# Query tweets
Load in tweets to MongoDB database  
*First start MongoDB service:*  
`mongod --config /usr/local/etc/mongod.conf`  
`brew services start mongodb-community`  

In [3]:
# Load dependencies
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
import os
import re
import pandas as pd

In [2]:
# Configuration
# Location MongoDB:
mongo_host = None
# Name collection
client_name = "fundamentals"

In [3]:
# Create connection
try:
    client = MongoClient(mongo_host)
    client.admin.command('ismaster')
    db = client[client_name] 
    twitter_db = db.twitter
    
except ConnectionFailure:
    print("Connection to MongoDB server could not be established")
    exit()

In [4]:
# Test connection by counting
print("Amount of tweets:",twitter_db.count_documents({}))

Amount of tweets: 657307


In [4]:
# Load in state election data
state_counts = pd.read_csv('../datasets/1976-2016-president.csv')

# Filter for 2016 election and the two major candidates
candidates = ['Clinton, Hillary', 'Trump, Donald J.']
state_2016 = state_counts[(state_counts["year"] == 2016)& 
                          (state_counts["candidate"].isin(candidates))]

# Sum candidate votes across multiple party names
state_candidates = state_2016[["state","candidate","candidatevotes"]].groupby(
    ["state","candidate"], as_index = True).sum()

# Create dataframe with relevant colums and merge with votecount
state_totals = state_2016[["state","state_po","candidate","totalvotes"]].drop_duplicates(
    subset=["state","candidate"])
election_totals = pd.merge(state_totals ,state_candidates, how = "right", 
                           on = ["state", "candidate"] )
election_totals["candidate_percent"] = election_totals["candidatevotes"] / election_totals["totalvotes"]


In [5]:
# Extract state name
# Create dataframe of state names
state_names = state_2016[["state","state_po"]].drop_duplicates().reset_index(drop=True)
state_names.to_pickle("../datasets/state_names.pkl")

In [7]:
def state_names_test(state_db, print_errors = False, loc_state_names: "../datasets/state_names.pkl"):
    state_names = pd.read_pickle(loc_state_names)
    n_errors = 0
    full_name_errors = dict()
    error_names = dict()
    n_correct = 0
    for obj in state_db:
        # Regex on last letters in place name and match to state
        state_short = re.findall(", ([A-Z]{2})$",obj["place"]['full_name'])
        if len(state_short) == 1:
            state_short = state_short[0]
            state_full = state_names[state_names["state_po"] == state_short].values[0,0]
            n_correct += 1
        # If no patten found search for state, USA    
        elif len(state_short) == 0:
            state_full = re.findall("([A-Za-z ]*\w+), USA$",obj["place"]['full_name'])
            if len(state_full) == 0:
                n_errors += 1
                error_empty = obj["place"]['full_name']
                if error_empty not in full_name_errors.keys():
                    full_name_errors[error_empty] = 1
                else:
                    full_name_errors[error_empty] = full_name_errors[error_empty] + 1
                continue
            elif len(state_full) == 1:
                state_full = state_full[0]
                try:
                    state_short = state_names[state_names["state"] == state_full].values[0,1]
                    n_correct += 1
                except:
                    error_value = obj["place"]['full_name']
                    n_errors += 1
                    if error_value not in error_names.keys():
                        error_names[error_value] = 1
                    else:
                        error_names[error_value] = error_names[error_value] + 1
                    continue
            else: print(obj["place"]['full_name'])
        if state_short == None:
            print(obj["place"]['full_name'], state_short, state_full)
    print("Correct: ", n_correct)
    print("Errors: ",n_errors)
    if print_errors:
        print("Full name")
        for name, value in full_name_errors.items():
            print(name,": ",value)
        print("Lookup")
        for name, value in error_names.items():
            print(name,": ",value)


In [8]:
# Test how many errors:
city_state = twitter_db.find(filter = {"place.country_code" : "US"}, projection = 
                           { "_id": 0 , "place.full_name": 1}, limit = 0)
state_names_test(city_state)

Correct:  587414
Errors:  5854


Less than 1% errors, not adding extra exceptions.

In [9]:
def state_names_json(state_json, print_errors = False
                     , loc_state_names: "../datasets/state_names.pkl"):
    state_names = pd.read_pickle(loc_state_names)
    tweet_list = list()
    n_errors = 0
    full_name_errors = dict()
    error_names = dict()
    n_correct = 0
    for obj in state_json:
        # Regex on last letters in place name and match to state
        state_short = re.findall(", ([A-Z]{2})$",obj["place"]['full_name'])
        if len(state_short) == 1:
            state_short = state_short[0]
            # Match abreviation with full state name
            state_full = state_names[state_names["state_po"] == state_short].values[0,0]
            n_correct += 1
        # If no patten found search for state, USA    
        elif len(state_short) == 0:
            state_full = re.findall("([A-Za-z ]*\w+), USA$",obj["place"]['full_name'])
            if len(state_full) == 0:
                n_errors += 1
                error_empty = obj["place"]['full_name']
                if error_empty not in full_name_errors.keys():
                    full_name_errors[error_empty] = 1
                else:
                    full_name_errors[error_empty] = full_name_errors[error_empty] + 1
                continue
            elif len(state_full) == 1:
                state_full = state_full[0]
                if sum(state_names["state"] == state_full) > 0:
                    # Match full state name with abreviation
                    state_short = state_names[state_names["state"] == state_full].values[0,1]
                    n_correct += 1
                else:
                    error_value = obj["place"]['full_name']
                    n_errors += 1
                    if error_value not in error_names.keys():
                        error_names[error_value] = 1
                    else:
                        error_names[error_value] = error_names[error_value] + 1
                    continue
            else: print(obj["place"]['full_name'])
        # Add state values to tweet and append to results
        
        obj["place"]["state_short"] = state_short
        obj["place"]["state_full"] = state_full
        tweet_list.append(obj)
    print("Correct: ", n_correct)
    print("Errors: ",n_errors)
    if print_errors:
        print("Full name")
        for name, value in full_name_errors.items():
            print(name,": ",value)
        print("Lookup")
        for name, value in error_names.items():
            print(name,": ",value)
    return(tweet_list)

In [10]:
# Example json list function:

city_state = twitter_db.find(filter = {"place.country_code" : "US"}, projection = 
                           { "_id": 0 , "text": 1, "place.full_name": 1}, limit = 500)
test_json = state_names_json(city_state)

Correct:  498
Errors:  2


In [11]:
def state_names_df(state_df, location_place = "place", dic_extracted = False,
                   print_errors = False, loc_state_names: "../datasets/state_names.pkl"):
    """
    Transforms the tweet locations to states values via regex lookup.
    Make sure state_names DataFrame is loaded.
    location_place: name of place column
    dic_extracted: Is the place value extracted from the dictionairy within the dataframe?
    print_errors: Print errors values
    """
    state_names = pd.read_pickle(loc_state_names)
    state_df["state_short"] = None
    state_df["state_full"] = None
    n_errors = 0
    full_name_errors = dict()
    error_names = dict()
    n_correct = 0
    for row in state_df.itertuples():
        # Regex on last letters in place name and match to state
        if dic_extracted:
            state_short = re.findall(", ([A-Z]{2})$",getattr(row,location_place))
        else:
            state_short = re.findall(", ([A-Z]{2})$",getattr(row,location_place)['full_name'])
        
        if len(state_short) == 1:
            state_short = state_short[0]
            # Match abreviation with full state name
            state_full = state_names[state_names["state_po"] == state_short].values[0,0]
            n_correct += 1

        # If no patten found search for state, USA    
        elif len(state_short) == 0:
            if dic_extracted:
                state_full = re.findall("([A-Za-z ]*\w+), USA$",getattr(row,location_place))
            else:
                state_full = re.findall("([A-Za-z ]*\w+), USA$",
                                        getattr(row,location_place)['full_name'])
            if len(state_full) == 0:
                n_errors += 1
                if dic_extracted:
                    error_empty = getattr(row,location_place)
                else:
                    error_empty = getattr(row,location_place)['full_name']
                if error_empty not in full_name_errors.keys():
                    full_name_errors[error_empty] = 1
                else:
                    full_name_errors[error_empty] = full_name_errors[error_empty] + 1
                continue
            elif len(state_full) == 1:
                state_full = state_full[0]
                if sum(state_names["state"] == state_full) > 0:
                    # Match full state name with abreviation
                    state_short = state_names[state_names["state"] == state_full].values[0,1]
                    n_correct += 1
                else:
                    if dic_extracted:
                        error_value = getattr(row,location_place)
                    else:
                        error_value = getattr(row,location_place)['full_name']
                    n_errors += 1
                    if error_value not in error_names.keys():
                        error_names[error_value] = 1
                    else:
                        error_names[error_value] = error_names[error_value] + 1
                    continue
            else: print("Error: ",getattr(row,location_place))
        # Add state values to correct column and append to results
        state_df.at[row[0], "state_short"] = state_short
        state_df.at[row[0], "state_full"] = state_full
    print("Correct: ", n_correct)
    print("Errors: ",n_errors)
    if print_errors:
        print("Full name")
        for name, value in full_name_errors.items():
            print(name,": ",value)
        print("Lookup")
        for name, value in error_names.items():
            print(name,": ",value)
    return(state_df)

In [12]:
# Example df function:
city_state = twitter_db.find(filter = {"place.country_code" : "US"}, projection = 
                           { "_id": 0 , "text": 1, "place.full_name": 1, 
                            "place.country_code" :1}, limit = 1000)
example_df = pd.DataFrame(city_state)
# example_df["test"] = None
# for row in example_df.itertuples():
#     example_df.at[row[0], "test"] = getattr(row,"place")['full_name']
test_example_df = state_names_df(example_df)
display(test_example_df)

Correct:  993
Errors:  7


,text,place,state_short,state_full
0,@theblaze @realDonaldTrump https://t.co/TY9DlZ...,"{'full_name': 'Frontenac, MO', 'country_code':...",MO,Missouri
1,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...,"{'full_name': 'Baton Rouge, LA', 'country_code...",LA,Louisiana
2,@theblaze @realDonaldTrump https://t.co/n050DB...,"{'full_name': 'Frontenac, MO', 'country_code':...",MO,Missouri
3,#CNN #newday clear #Trump deliberately throwin...,"{'full_name': 'Baltimore, MD', 'country_code':...",MD,Maryland
4,"@realDonaldTrump, you wouldn't recognize a lie...","{'full_name': 'Palm Springs, CA', 'country_cod...",CA,California
...,...,...,...,...
995,@realDonaldTrump You Sir are a mental case !\n...,"{'full_name': 'Doylestown, PA', 'country_code'...",PA,Pennsylvania
996,"#inequality is not #democracy,its man made by ...","{'full_name': 'New Jersey, USA', 'country_code...",NJ,New Jersey
997,@realDonaldTrump The president of #isis says y...,"{'full_name': 'Indio, CA', 'country_code': 'US'}",CA,California
998,@sethmoulton Stop trying to DEFEND the INDEFEN...,"{'full_name': 'New Jersey, USA', 'country_code...",NJ,New Jersey


In [13]:
test_example_df[test_example_df["state_full"].isna()].head()

,text,place,state_short,state_full
408,@magnifier661 @SallyAu81783497 @HillaryClinton...,"{'full_name': 'Tumon, USA', 'country_code': 'US'}",None,None
460,@magnifier661 @SallyAu81783497 @HillaryClinton...,"{'full_name': 'Tumon, USA', 'country_code': 'US'}",None,None
525,@realDonaldTrump is definitely going to beat O...,"{'full_name': 'Allendale, Austin', 'country_co...",None,None
577,@Dollfinish @realDonaldTrump @EnemyWithinn @el...,"{'full_name': 'Tumon, USA', 'country_code': 'US'}",None,None
589,"@realDonaldTrump ""Trump acknowledges he could ...","{'full_name': 'Estados Unidos', 'country_code'...",None,None


In [63]:
# Create pickle of DataFrame of all us tweets with added state locations
print("Number of US country tweets", twitter_db.count_documents(filter = 
 {"place.country_code" : "US"}))
total_df = pd.DataFrame(twitter_db.find(filter = {"place.country_code" : "US"}, limit = 0))

total_df.to_pickle("dataset/state_tweet.pkl")

Number of US country tweets 593268


In [68]:
# MongoDB pipeline
# Run once to set database
state_names = pd.read_pickle("../datasets/state_names.pkl")
for state in state_names.itertuples():
    result = twitter_db.update_many(filter = {"$and" : [{"place.country_code" : "US"},
                {"$or": [{"place.full_name" : {"$regex" :", "+state.state_po+"$"}},
                         {"place.full_name" : {"$regex" :state.state+", USA$"}}
                                    ]}]},
                          update = {"$set": { "place.state" : state.state }})
    print("State: "+state.state+", Matched: "+str(result.matched_count)+", Modified: "+ 
          str(result.modified_count))


State: Alabama, Matched: 8194, Modified: 0
State: Alaska, Matched: 1399, Modified: 0
State: Arizona, Matched: 17687, Modified: 0
State: Arkansas, Matched: 3293, Modified: 0
State: California, Matched: 72446, Modified: 0
State: Colorado, Matched: 9511, Modified: 0
State: Connecticut, Matched: 5075, Modified: 0
State: Delaware, Matched: 1606, Modified: 0
State: District of Columbia, Matched: 4452, Modified: 0
State: Florida, Matched: 57823, Modified: 0
State: Georgia, Matched: 19105, Modified: 0
State: Hawaii, Matched: 2773, Modified: 0
State: Idaho, Matched: 1955, Modified: 0
State: Illinois, Matched: 22042, Modified: 0
State: Indiana, Matched: 7486, Modified: 0
State: Iowa, Matched: 4280, Modified: 0
State: Kansas, Matched: 3434, Modified: 0
State: Kentucky, Matched: 12200, Modified: 0
State: Louisiana, Matched: 9852, Modified: 0
State: Maine, Matched: 4469, Modified: 0
State: Maryland, Matched: 11109, Modified: 0
State: Massachusetts, Matched: 13388, Modified: 0
State: Michigan, Match

In [69]:
print(twitter_db.count_documents({ "place.state":{"$exists" : True} }))

587414


In [ ]:
# Example MongoDB queries
# Distinct languages
twitter_db.distinct("lang")

# Extract data form bounding box
location = twitter_db.find(filter = {"place.country_code" : "US"}, projection = 
                           { "_id": 0 , "place.bounding_box.coordinates": 1}, limit = 5)
for obj in location:
    print(obj["place"]['bounding_box']['coordinates'])